In [1]:
import requests
from bs4 import BeautifulSoup
import spacy
import json
import re

!pip install spacy-entity-linker
!python -m spacy_entity_linker "download_knowledge_base"

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for spacy-entity-linker: filename=spacy_entity_linker-1.0.3-py3-none-any.whl size=14393 sha256=af0716bdbe13a100ed1e3e07e1f099e3bfc201c4dc64288898a6d3bb3da31a75
  Stored in directory: /root/.cache/pip/wheels/46/56/da/45f697f5dd8163f7afda834ced7da52e36e45d43c44a61db84
Successfully built spacy-entity-linker
/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-19 18:07:13.781270: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-19 18:07:13.781457: W tensorflow/compiler/xla/stream_executor/platform/default/dso_l

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

DRIVE_PATH = "/content/drive/MyDrive"

Mounted at /content/drive/


In [5]:
def clean_document(text):
  if text.find("’"):
    text = text.replace("’", "'")
  text = re.sub(r"won\'t", "will not", text)
  text = re.sub(r"can\'t", "can not", text)
  text = re.sub(r"in\'", "ing", text)
  text = re.sub(r"\'em", "them", text)
  text = re.sub(r"n\'t", " not", text)
  text = re.sub(r"\'re", " are", text)
  text = re.sub(r"\'s", " is", text)
  text = re.sub(r"\'d", " would", text)
  text = re.sub(r"\'ll", " will", text)
  text = re.sub(r"\'t", " not", text)
  text = re.sub(r"\'ve", " have", text)
  text = re.sub(r"\'m", " am", text)
  text = " ".join(text.split())
  return text

In [6]:
list_files = !ls /content/drive/MyDrive/IE/ie_course_2022_team03/retrieved_data/scrapped_data

In [10]:
# nlp = spacy.load("en_core_web_md")

# nlp.add_pipe("entityLinker", last=True)

qid, unqiue_qids = [], []

for file in list_files:
  if len(unqiue_qids) > 600:
    break
  with open(DRIVE_PATH + f"/IE/ie_course_2022_team03/retrieved_data/scrapped_data/{file}") as f:
    file_data = json.load(f)
    textual_data =  " ".join(file_data['text'])
    processed_data = clean_document(textual_data)
    doc = nlp(processed_data)
    all_linked_entities = doc._.linkedEntities
    for i in all_linked_entities:
      qid.append("Q"+str(i.get_id()))
    unqiue_qids = [*set(qid)]
print(unqiue_qids)
with open(f'{DRIVE_PATH}/IE/ie_course_2022_team03/assets/Qids.json', 'w') as file:
  json.dump(unqiue_qids,file,indent=4)

JSONDecodeError: ignored

In [ ]:
entity_list = []
for id in unqiue_qids:
  try:
    root_url = "https://www.wikidata.org/wiki/"+id
    response= requests.get(root_url)
    d={}
    soup = BeautifulSoup(response.content,"html.parser")
    item = soup.find("div",{"id": "content"})
    qid = item.find("span",{"class":"wikibase-title-id"})
    d["qid"] = qid.get_text().replace("(","").replace(")","")
    d["name"] = item.find("span",{"class":"wikibase-title-label"}).get_text()
    if(item.find("div",{"class":"wikibase-aliasesview "}) == None):
      d["aliases"] = ""
    else:
      d["aliases"]= item.find("div",{"class":"wikibase-aliasesview "}).get_text(separator="\n",strip=True).split("\n")

    d["description"] = str(item.find("div",{"class":"wikibase-entitytermsview-heading-description "}).get_text())
    d["label"] = ' '.join([i.label_ for i in nlp(d["name"]).ents])
    json_object = json.dumps(d, indent = 4) 
    entity_list.append(d)
    print("Data stored with Qid ", id)
  except Exception as e:
    print(e)

print(entity_list)
print(len(entity_list))
name_list = list(i['name'] for i in entity_list)
alias_list = list(i['aliases'] for i in entity_list)
alias_list = [item for sublist in alias_list for item in sublist]
gazetteers_list=name_list+alias_list
print(gazetteers_list)

In [ ]:
with open(f'{DRIVE_PATH}/IE/ie_course_2022_team03/assets/entities.json', 'w') as file:
  json.dump(entity_list,file,indent=4)
with open(f'{DRIVE_PATH}/IE/ie_course_2022_team03/assets/entities_gazetteers.json', 'w') as file:
  json.dump(gazetteers_list,file,indent=4)